# Example of pipeline: extracting and transforming pdf file

In this example, we will show you how to end-to-end generate question-answers (QAs) from a given pdf using uniflow's `MultiFlowsPipeline`.

### Before running the code

You will need to `uniflow` conda environment to run this notebook. You can set up the environment following the instruction: https://github.com/CambioML/uniflow/tree/main#installation.

### Update system path

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import pprint
import re

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

### Install libraries

In [2]:
!{sys.executable} -m pip install transformers accelerate bitsandbytes scipy nougat-ocr

### Import dependency

In [3]:
import os
import pandas as pd
from uniflow.pipeline import MultiFlowsPipeline
from uniflow.flow.config import PipelineConfig
from uniflow.flow.config import TransformHuggingFaceConfig, ExtractPDFConfig
from uniflow.op.model.model_config import HuggingfaceModelConfig, NougatModelConfig
from uniflow.op.prompt_schema import GuidedPrompt, Context
from uniflow.op.extract.split.constants import MARKDOWN_HEADER_SPLITTER

/opt/conda/envs/uniflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Prepare the input data

First, let's set current directory and input data directory, and load the raw data.

In [4]:
dir_cur = os.getcwd()
pdf_file = "nike-paper.pdf"
input_file = os.path.join(f"{dir_cur}/data/raw_input/", pdf_file)

data = [
    {"pdf": input_file},
]

### Define extract config using Nougat

In [5]:
extract_config = ExtractPDFConfig(
    model_config=NougatModelConfig(
        model_name = "0.1.0-small",
        batch_size = 1 # When batch_size>1, nougat will run on CUDA, otherwise it will run on CPU
    ),
    splitter=MARKDOWN_HEADER_SPLITTER
)

### Prepare sample prompts

We need to demonstrate sample prompts for LLM. We do this by giving a sample list of `Context` examples to the `GuidedPrompt` class.

In [6]:
guided_prompt = GuidedPrompt(
    instruction="""Generate one question and its corresponding answer based on the last context in the last
    example. Follow the format of the examples below to include context, question, and answer in the response""",
    examples=[
        Context(
            context="In 1948, Claude E. Shannon published A Mathematical Theory of\nCommunication (Shannon, 1948) establishing the theory of\ninformation. In his article, Shannon introduced the concept of\ninformation entropy for the first time. We will begin our journey here.",
            question="Who published A Mathematical Theory of Communication in 1948?",
            answer="Claude E. Shannon.",
        ),
])

### Define transform config

In this example, we will use the [HuggingfaceModelConfig](https://github.com/CambioML/uniflow/blob/main/uniflow/model/config.py#L39)'s default LLM to generate questions and answers. Let's import the config of this model.

Here, we pass in our `guided_prompt` to the `HuggingfaceConfig` to use our customized instructions and examples, instead of the `uniflow` default ones.

Note, base on your GPU memory, you can set your optimal `batch_size` below. 

In [7]:
current_batch_size = 1
print("batch size:", current_batch_size)

transform_config = TransformHuggingFaceConfig(
    guided_prompt_template=guided_prompt,
    model_config=HuggingfaceModelConfig(batch_size=current_batch_size)
)

batch size: 1


### Use MultiFlowsPipeline

Let's import the `PipelineConfig` of `MultiFlowsPipeline` to connect `extract_config` and `transform_config`.

In [8]:
p = MultiFlowsPipeline(PipelineConfig(
    extract_config=extract_config,
    transform_config=transform_config,
))

/opt/conda/envs/uniflow/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.16s/it]


Now we call the `run` method on the `MultiFlowsPipeline` object to execute the question-answer generation operation on the data shown above.

In [9]:
output = p.run(data)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]/opt/conda/envs/uniflow/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:228: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
 77%|███████▋  | 10/13 [02:09<00:53, 17.88s/it]/opt/conda/envs/uniflow/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 13/13 [02:33<00:00, 11.79s/it]


### Output

Let's take a look of the generated output of Abstract segmentation.

In [11]:
pprint.pprint(output[0][1]['output'][0]['response'][0])

('instruction: Generate one question and its corresponding answer based on the '
 'last context in the last\n'
 '    example. Follow the format of the examples below to include context, '
 'question, and answer in the response\n'
 'context: In 1948, Claude E. Shannon published A Mathematical Theory of\n'
 'Communication (Shannon, 1948) establishing the theory of\n'
 'information. In his article, Shannon introduced the concept of\n'
 'information entropy for the first time. We will begin our journey here.\n'
 'question: Who published A Mathematical Theory of Communication in 1948?\n'
 'answer: Claude E. Shannon.\n'
 'context: ###### Abstract\n'
 'We collected marathon performance data from a systematic sample of elite and '
 'sub-elite athletes over the period 2015 to 2019, then searched the internet '
 'for publicly-available photographs of these performances, identifying '
 'whether the Nike Vaporly shoes were worn or not in each performance. '
 'Controlling for athlete ability and ra

### Process the output

Let's take a look of the generated output. We need to do a little postprocessing on the raw output.

In [43]:
# Extracting context, question, and answer into a DataFrame
contexts = []
questions = []
answers = []

keywords = ["context:", "question:", "answer:"]
pattern = '|'.join(map(re.escape, keywords))

for item in output[0]:
    o = item['output'][0]['response'][0]
    segments = [segment for segment in re.split(pattern, o) if segment.strip()]

    contexts.append(segments[-3])
    questions.append(segments[-2])
    answers.append(segments[-1])

# Set display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)

df = pd.DataFrame({
    'Context': contexts,
    'Question': questions,
    'Answer': answers
})

styled_df = df.style.set_properties(**{'text-align': 'left'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'left')]
}])
styled_df